<a href="https://colab.research.google.com/github/Kevinlo937/cord-cutting-predicated/blob/main/topMSO_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Load the required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Plot styling
import seaborn as sns; # sns.set()  # for plot styling
%matplotlib inline

import os
import glob

參考以下指令，自同一個目錄下，取得所有 .csv 檔案，將內容併入同一dataset

Load dataset dataset=pd.read_csv('/content/drive/Othercomputers/我的筆記型電腦/ExpertBook/2025/用戶資料集/過濾資料/用戶服務資料/用戶15000.csv', sep='^')

In [3]:
# 設定目錄路徑
directory_path = '/content/drive/Othercomputers/我的筆記型電腦/ExpertBook/2025/用戶資料集/過濾資料/用戶服務資料2/'

# 取得所有 .csv 檔案的路徑
all_files = glob.glob(os.path.join(directory_path, "*.csv"))

# 建立一個空的 list 來存放所有 DataFrame
all_dataframes = []

# 迭代所有檔案路徑，讀取 CSV 並加入 list
for filename in all_files:
    df = pd.read_csv(filename, sep='^')
    all_dataframes.append(df)

# 使用 pd.concat 合併所有 DataFrame
dataset = pd.concat(all_dataframes, ignore_index=True)

In [4]:
dataset.head()

,客編,產品名稱,用戶種類,相關編號,起日,迄日,系統台,地區,用戶類別,繳別,使用狀態
0,503,EPON,一般收視戶,2569881,2023/04/20 11:17:58,NaN,大屯,大里區,一般收視戶,1,使用中
1,503,LINE TV,大樓集收,2504415,2022/08/11 10:49:51,NaN,大屯,大里區,大樓集收,6,停用
2,503,EPON,大樓集收,2504414,2022/08/11 10:49:29,NaN,大屯,大里區,大樓集收,6,停用
3,503,DTV,大樓集收,1017488,2012/05/16 11:38:20,NaN,大屯,大里區,大樓集收,12,停用
4,503,CATV,大樓集收,708,2006/05/01 00:00:00,2006/05/01 00:00:00,大屯,大里區,大樓集收,1,停用


In [5]:
# '用戶類別' 欄位與 '用戶種類' 之表達重複，清除刪掉該欄位
dataset = dataset.drop(columns=['用戶類別'])
dataset.head()

,客編,產品名稱,用戶種類,相關編號,起日,迄日,系統台,地區,繳別,使用狀態
0,503,EPON,一般收視戶,2569881,2023/04/20 11:17:58,NaN,大屯,大里區,1,使用中
1,503,LINE TV,大樓集收,2504415,2022/08/11 10:49:51,NaN,大屯,大里區,6,停用
2,503,EPON,大樓集收,2504414,2022/08/11 10:49:29,NaN,大屯,大里區,6,停用
3,503,DTV,大樓集收,1017488,2012/05/16 11:38:20,NaN,大屯,大里區,12,停用
4,503,CATV,大樓集收,708,2006/05/01 00:00:00,2006/05/01 00:00:00,大屯,大里區,1,停用


In [6]:
# 1. 統計 '用戶種類' 的分類數量
user_type_counts = dataset['用戶種類'].value_counts()

# 2. 顯示統計結果
print(user_type_counts)

用戶種類
一般收視戶      2129797
大樓集收        256772
大樓個收        171527
類有線          46516
公播戶          25350
            ...   
電器行             21
警消專案            20
電信單位            15
優待戶(全優)          8
長繳別              1
Name: count, Length: 75, dtype: int64


In [7]:
# Keep only rows where user_type is "Regular Subscribers"
dataset = dataset[dataset['用戶種類'] == "一般收視戶"]

# Optional: Reset the index after removing rows
dataset = dataset.reset_index(drop=True)

In [8]:
# 1. Count the occurrences of each service status
service_status_counts = dataset['使用狀態'].value_counts()

# 2. Display the counts
print(service_status_counts)

使用狀態
停用            1185652
使用中            864587
拆機中             39591
無法完工            16659
欠款斷線            12846
維修中              5274
暫停               4010
設備加裝              998
移機中                87
派收中                58
作業中                14
裝機中                 9
促銷中                 4
重複註銷                3
復機中                 2
作廢                  2
退租(先斷線未拆機)          1
Name: count, dtype: int64


In [9]:
# 1. Group by '客編' and '相關編號' and count occurrences
service_counts_per_customer = dataset.groupby(['客編', '相關編號']).size().reset_index(name='counts')

# 2. Group by '客編' and get the number of unique '相關編號' for each '客編'
unique_service_counts = service_counts_per_customer.groupby('客編')['相關編號'].nunique().reset_index(name='unique_service_count')

# 3. Analyze the distribution of unique service counts
print(unique_service_counts['unique_service_count'].describe())

count    833464.000000
mean          2.555356
std           2.300758
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max         610.000000
Name: unique_service_count, dtype: float64


In [10]:
# print(unique_service_counts)
specific_count_value = 100  # Replace with the actual customer ID
customer_data = unique_service_counts[unique_service_counts['unique_service_count'] >= specific_count_value]
print(customer_data)

             客編  unique_service_count
572071   822687                   148
577211   828488                   424
616666   880611                   360
616690   880793                   421
617612   882431                   179
623898   893351                   610
639463   920197                   121
670969   978468                   146
677889   996540                   125
693886  1045629                   135
701683  1058814                   210
769668  1156512                   232


In [11]:
specific_customer_id = 822687                     # Replace with the actual customer ID
customer_data = dataset[dataset['客編'] == specific_customer_id]
print(customer_data)

             客編  產品名稱   用戶種類     相關編號                   起日  \
1510991  822687  電路出租  一般收視戶   897381  2010/07/14 00:00:00   
1510992  822687  電路出租  一般收視戶   897372  2010/07/14 00:00:00   
1514293  822687  電路出租  一般收視戶   928444  2011/02/21 00:00:00   
1514294  822687  電路出租  一般收視戶   897385  2010/07/14 00:00:00   
1514295  822687  電路出租  一般收視戶   897366  2009/07/07 00:00:00   
...         ...   ...    ...      ...                  ...   
1514526  822687  電路出租  一般收視戶  2413581  2021/08/02 09:00:19   
1514527  822687  電路出租  一般收視戶  2421817  2021/09/07 17:22:26   
1514528  822687  電路出租  一般收視戶  2712557  2024/08/16 16:54:29   
1514529  822687  電路出租  一般收視戶  2712555  2024/08/16 16:52:31   
1514530  822687  電路出租  一般收視戶  2712556  2024/08/16 16:53:34   

                          迄日 系統台   地區  繳別 使用狀態  
1510991  2010/02/02 00:00:00  中投  南投市   1   停用  
1510992  2009/12/30 00:00:00  中投  南投市   1   停用  
1514293  2011/02/21 00:00:00  中投  南投市   1   停用  
1514294  2010/02/02 00:00:00  中投  草屯鎮   1   停用  
1514295  2

In [12]:
# 1. 統計 '用戶種類' 的分類數量
product_type_counts = dataset['產品名稱'].value_counts()

# 2. 顯示統計結果
print(product_type_counts)

產品名稱
DTV              888046
CATV             771864
CM               231239
EPON             110778
LINE TV          105253
電路出租               5748
熊搭心(瑪帛)            5374
LiTV               2822
HBOGO              2319
MESH WIFI加值服務      1752
居家智慧攝影機            1512
零信任                1227
Friday影音            645
CatchPlay+          339
宅健康                 303
Ibiza成人             148
點晴-智慧健身             141
統簽網路管理               80
美華卡拉OK               58
VOD影音隨選              40
Ugamenow             19
行銷方案                 16
VOIP                 14
瑪帛視訊                  9
智慧家庭                  9
監視器方案                 8
PILI線上看               7
wifi上網時間管理            7
聰明象快樂學堂               5
迎賓服務                  5
黃背心                   5
FB社群wifi              3
電視隨身看                 2
Name: count, dtype: int64


In [13]:
# 將 dataset 中 '產品名稱' 欄位值為 '電路出租' 的資料 drop 掉
dataset = dataset[dataset['產品名稱'] != '電路出租']
dataset = dataset.reset_index(drop=True)

In [14]:
dataset.to_csv('cleaned_dataset2.csv', index=False, sep='^')

參考以下指令，自同一個目錄下，取得所有檔名以 '工單_*' 開頭的所有.csv 檔案，將內容併入同一 ds_doSevice 資料集

Load 工單 dataset ds_doService=pd.read_csv('/content/drive/Othercomputers/我的筆記型電腦/ExpertBook/2025/用戶資料集/過濾資料/工單/工單_20220201_20220301.csv', sep='^')

In [15]:
# 設定目錄路徑
directory_path = '/content/drive/Othercomputers/我的筆記型電腦/ExpertBook/2025/用戶資料集/過濾資料/工單/'

# 取得所有檔名以 '工單_2022' 開頭的 .csv 檔案的路徑
all_files = glob.glob(os.path.join(directory_path, "工單_*.csv"))

# 建立一個空的 list 來存放所有 DataFrame
all_dataframes = []

# 迭代所有檔案路徑，讀取 CSV 並加入 list
for filename in all_files:
    df = pd.read_csv(filename, sep='^')
    all_dataframes.append(df)

# 使用 pd.concat 合併所有 DataFrame
ds_doService = pd.concat(all_dataframes, ignore_index=True)


In [16]:
ds_doService.head()

,客編,相關編號,工單日期,完工日期,預約日期,工單原因
0,627459,2133530,2021/12/28 14:59:22,2022/02/05 23:38:29,2022/02/06 15:30:00,STB操作不熟導致
1,1141908,2427350,2021/12/30 11:01:05,2022/02/05 08:27:42,2022/02/06 12:00:00,工程到時故障已排除
2,671764,2448261,2021/12/28 19:23:16,2022/02/07 18:05:09,2022/02/07 08:00:00,其他_CM轉EPON
3,1018796,1857860,2022/01/07 10:07:56,2022/02/05 23:39:59,2022/02/06 12:00:00,協助解決用戶設備(電視、音響)
4,622924,1088181,2022/01/06 11:55:33,2022/02/07 13:40:18,2022/02/07 12:00:00,EPON設定_人為問題-ONU設定異常


In [17]:
# Calculate the time difference in hours
ds_doService['等待時間'] = (pd.to_datetime(ds_doService['完工日期'], format='%Y/%m/%d %H:%M:%S', errors='coerce').astype('int64') // 10**9 - pd.to_datetime(ds_doService['工單日期'], format='%Y/%m/%d %H:%M:%S').astype('int64') // 10**9) / 3600
# Convert '等待時間' to days
ds_doService['等待天數'] = ds_doService['等待時間'] / 24  # Divide by 24 to convert hours to days
ds_doService['等待天數'] = ds_doService['等待天數'].round()

In [18]:
# 針對 ds_doService['等待天數'] 的值做分布統計

# 1. 統計每個唯一值的出現次數
waiting_days_counts = ds_doService['等待天數'].value_counts()

# 2. 顯示統計結果
print(waiting_days_counts)

等待天數
0.0      397611
1.0      172526
2.0       25253
3.0       11753
4.0        6898
          ...  
200.0         1
87.0          1
79.0          1
98.0          1
96.0          1
Name: count, Length: 133, dtype: int64


In [19]:
ds_doService.to_csv('cleaned_ds_doService.csv', index=False, sep='^')

In [20]:
# 1. Count the occurrences of each unique '客編'
# customer_counts = ds_doService['客編'].value_counts()

# 2025/02/01 : 因為客編與相關編號(服務編號)為1對多關係，而工單紀錄實際上以相關編號為主
# ，因此觀察多次派工行為時先以相關編號進行統計，必要時再關聯回客編，以避免同一客編的不同
# 服務次數被累加至一起
customer_counts = ds_doService['相關編號'].value_counts()
# Count the occurrences of each unique count value
count_distribution = customer_counts.value_counts()
print(customer_counts)

相關編號
0          458
1357838     58
2554076     55
2248665     43
1418365     41
          ... 
2605536      1
1550401      1
1561826      1
1979195      1
1854211      1
Name: count, Length: 345965, dtype: int64


In [21]:
specific_customer_id = 0                     # Replace with the actual customer ID
customer_data = ds_doService[ds_doService['相關編號'] == specific_customer_id]
print(customer_data)

             客編  相關編號                 工單日期                 完工日期  \
448       88768     0  2022/02/24 17:09:19  2022/02/25 20:31:19   
702     1039601     0  2022/02/17 08:54:42  2022/02/17 09:53:58   
1619     769816     0  2022/02/20 20:21:14  2022/02/21 15:40:45   
1778     900195     0  2022/02/19 17:36:28  2022/02/20 10:42:49   
2973     960865     0  2022/02/28 09:49:53  2022/02/28 13:21:50   
...         ...   ...                  ...                  ...   
632012   410242     0  2025/01/20 17:01:52  2025/01/20 17:05:42   
632023   357706     0  2025/01/13 12:30:10  2025/01/13 15:41:21   
632046  1230453     0  2025/01/19 15:03:57  2025/01/20 10:16:09   
633835   331413     0  2025/01/09 08:33:03  2025/01/09 18:04:24   
637126  1168225     0  2025/01/16 16:23:19  2025/01/17 12:48:38   

                       預約日期                    工單原因       等待時間  等待天數  
448     2022/02/25 18:30:00                   更換遙控器  27.366667   1.0  
702     2022/02/17 12:00:00                    固定纜線  

服務編號(相關編號)應為唯一，但針對 ‘相關編號’ 進行次數統計時，發現有168筆相關編號為 0 的紀錄，且對應至不同 ‘客編’ --> drop

In [22]:
ds_doService=ds_doService[ds_doService['相關編號']!=0]

In [24]:
ds_doService.head()

,客編,相關編號,工單日期,完工日期,預約日期,工單原因,等待時間,等待天數
0,627459,2133530,2021/12/28 14:59:22,2022/02/05 23:38:29,2022/02/06 15:30:00,STB操作不熟導致,944.651944,39.0
1,1141908,2427350,2021/12/30 11:01:05,2022/02/05 08:27:42,2022/02/06 12:00:00,工程到時故障已排除,885.443611,37.0
2,671764,2448261,2021/12/28 19:23:16,2022/02/07 18:05:09,2022/02/07 08:00:00,其他_CM轉EPON,982.698056,41.0
3,1018796,1857860,2022/01/07 10:07:56,2022/02/05 23:39:59,2022/02/06 12:00:00,協助解決用戶設備(電視、音響),709.534167,30.0
4,622924,1088181,2022/01/06 11:55:33,2022/02/07 13:40:18,2022/02/07 12:00:00,EPON設定_人為問題-ONU設定異常,769.745833,32.0


In [23]:
# 以 ds_doService 服務編號所對應 dataset 中服務編號的使用狀態, 加到 ds_doService_2 資料集中
ds_doService_2 = pd.merge(ds_doService, dataset[['客編', '相關編號', '使用狀態']], on=['客編', '相關編號'], how='left')

In [25]:
ds_doService_2.head()

,客編,相關編號,工單日期,完工日期,預約日期,工單原因,等待時間,等待天數,使用狀態
0,627459,2133530,2021/12/28 14:59:22,2022/02/05 23:38:29,2022/02/06 15:30:00,STB操作不熟導致,944.651944,39.0,使用中
1,1141908,2427350,2021/12/30 11:01:05,2022/02/05 08:27:42,2022/02/06 12:00:00,工程到時故障已排除,885.443611,37.0,使用中
2,671764,2448261,2021/12/28 19:23:16,2022/02/07 18:05:09,2022/02/07 08:00:00,其他_CM轉EPON,982.698056,41.0,使用中
3,1018796,1857860,2022/01/07 10:07:56,2022/02/05 23:39:59,2022/02/06 12:00:00,協助解決用戶設備(電視、音響),709.534167,30.0,NaN
4,622924,1088181,2022/01/06 11:55:33,2022/02/07 13:40:18,2022/02/07 12:00:00,EPON設定_人為問題-ONU設定異常,769.745833,32.0,NaN


In [26]:
# 依 ds_doService 中客編找出不存在 dataset中的資料，存入 ds_noExist 資料集
ds_noExist = ds_doService[~ds_doService['客編'].isin(dataset['客編'])]

In [27]:
ds_noExist.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61643 entries, 3 to 637148
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   客編      61643 non-null  int64  
 1   相關編號    61643 non-null  int64  
 2   工單日期    61643 non-null  object 
 3   完工日期    61643 non-null  object 
 4   預約日期    61643 non-null  object 
 5   工單原因    61643 non-null  object 
 6   等待時間    61643 non-null  float64
 7   等待天數    61643 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 4.2+ MB


In [28]:
# 刪掉 ds_doService_2 中客編存在 ds_noExist 客編鐘資料
ds_doService_2 = ds_doService_2[~ds_doService_2['客編'].isin(ds_noExist['客編'])]

In [29]:
# 刪掉 ds_doService_2 中使用狀態為空值資料
ds_doService_2 = ds_doService_2.dropna(subset=['使用狀態'])

In [30]:
ds_doService_2.to_csv('cleaned_ds_doService.csv', index=False, sep='^')

In [35]:
# ds_doService 為工單資料集，針對同一 '相關編號' 存在多筆工單紀錄時，比較其工單日期，以時間最末的工單日期為基礎，分別計算工單日期與其相距 30天內、60天內、90天內、以及90天以上的工單數量，並隨著相關編號存入 serviceDay_diff 資料集
# import pandas as pd

def calculate_service_day_diff(ds_doService):
    """
    Calculates the number of service orders within different time ranges for each '相關編號',
    accumulating counts based on where time_diffs fall within intervals.

    Args:
        ds_doService (pd.DataFrame): The service order dataset.

    Returns:
        pd.DataFrame: A DataFrame with '相關編號' and service order counts within time ranges.
    """

    # Group by '相關編號' and get the latest '工單日期' for each group
    latest_dates = ds_doService.groupby('相關編號')['工單日期'].max()

    # Create an empty dictionary to store the results
    service_day_diff = {}

    # Iterate through each unique '相關編號'
    for customer_id, latest_date in latest_dates.items():
        # Get all service orders for the current customer
        customer_orders = ds_doService[ds_doService['相關編號'] == customer_id]

        # Calculate the time difference between each order date and the latest date
        time_diffs = (latest_date - customer_orders['工單日期']).dt.days

        # Initialize counts for each range
        within_30_days = 0
        within_60_days = 0
        within_90_days = 0
        over_90_days = 0

        # Accumulate counts based on where time_diffs fall
        for diff in time_diffs:
            if diff <= 30:
                within_30_days += 1
            elif diff <= 60:
                within_60_days += 1
            elif diff <= 90:
                within_90_days += 1
            else:
                over_90_days += 1

        # Store the counts in the dictionary
        service_day_diff[customer_id] = {
            '工單日期': latest_date,
            '30天內': within_30_days,
            '60天內': within_60_days,
            '90天內': within_90_days,
            '90天以上': over_90_days
        }

    # Convert the dictionary to a DataFrame
    service_day_diff_df = pd.DataFrame.from_dict(service_day_diff, orient='index')
    service_day_diff_df.index.name = '相關編號'

    return service_day_diff_df

In [36]:
# Assuming ds_doService is your DataFrame:
serviceDay_diff = calculate_service_day_diff(ds_doService_2)

In [37]:
serviceDay_diff.head()

,工單日期,30天內,60天內,90天內,90天以上
相關編號,,,,,
16,2023-01-23 15:18:21,1,0,0,0
19,2024-08-13 21:53:11,1,0,0,1
39,2023-02-15 10:51:09,1,0,0,0
67,2022-04-15 10:47:15,1,0,0,0
149,2023-03-20 11:37:23,1,0,0,0


In [40]:
serviceDay_diff.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300687 entries, 16 to 2749539
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   工單日期    300687 non-null  datetime64[ns]
 1   30天內    300687 non-null  int64         
 2   60天內    300687 non-null  int64         
 3   90天內    300687 non-null  int64         
 4   90天以上   300687 non-null  int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 13.8 MB


In [38]:
serviceDay_diff.to_csv('serviceDay_diff.csv', index=False, sep='^')

In [39]:
# ds_doService 為工單資料集，計算同一 '相關編號' 所對應 '等待天數'，計算'等待天數'的平均值，並隨同 '相關編號' 存入 aveWaittingDay
# Group data by '相關編號' and calculate the mean of '等待天數'
aveWaittingDay = ds_doService_2.groupby('相關編號')['等待天數'].mean()

# Reset the index to make '相關編號' a regular column
aveWaittingDay = aveWaittingDay.reset_index()

# Rename the column (optional) for better readability
aveWaittingDay = aveWaittingDay.rename(columns={'等待天數': '平均等待天數'})

# Display the result
print(aveWaittingDay)


           相關編號  平均等待天數
0            16     0.0
1            19     0.0
2            39     1.0
3            67     0.0
4           149     0.0
...         ...     ...
300682  2749281     0.0
300683  2749312     0.0
300684  2749382     1.0
300685  2749524     0.0
300686  2749539     0.0

[300687 rows x 2 columns]


In [41]:
# 依 '相關編號' 合併 serviceDay_diff 、aveWaittingDay，並加入ds_doService中同一 相關編號所屬 客編、相關編號，存入 ds_doService_combo 資料集
# Merge serviceDay_diff and aveWaittingDay
merged_df = pd.merge(serviceDay_diff, aveWaittingDay, on='相關編號', how='left')

# Get 客編 and 使用狀態 from ds_doService
customer_info = ds_doService_2[['客編', '相關編號']].drop_duplicates(subset=['相關編號'])

# Merge with the previous result
ds_doService_combo = pd.merge(customer_info, merged_df, on='相關編號', how='left')

print(ds_doService_combo)

             客編     相關編號                工單日期  30天內  60天內  90天內  90天以上  平均等待天數
0            11       16 2023-01-23 15:18:21     1     0     0      0     0.0
1            13       19 2024-08-13 21:53:11     1     0     0      1     0.0
2            28       39 2023-02-15 10:51:09     1     0     0      0     1.0
3            47       67 2022-04-15 10:47:15     1     0     0      0     0.0
4        790667      149 2023-03-20 11:37:23     1     0     0      0     0.0
...         ...      ...                 ...   ...   ...   ...    ...     ...
300682  1266944  2749281 2025-01-20 14:50:52     1     0     0      0     0.0
300683  1267096  2749312 2025-01-23 15:47:03     1     0     0      0     0.0
300684  1266721  2749382 2025-01-22 13:27:55     1     0     0      0     1.0
300685  1266786  2749524 2025-01-23 13:40:59     1     0     0      0     0.0
300686  1266951  2749539 2025-01-22 09:51:19     1     0     0      0     0.0

[300687 rows x 8 columns]


In [42]:
# 以 ds_doService_combo 中的 客編與相關編號為基礎，將 dataset 的其他欄位資料合併至ds_doService_combo
# Merge ds_doService_combo with dataset based on '客編' and '相關編號'
ds_doService_combo = pd.merge(ds_doService_combo, dataset, on=['客編', '相關編號'], how='left')

In [43]:
ds_doService_combo.head()

,客編,相關編號,工單日期,30天內,60天內,90天內,90天以上,平均等待天數,產品名稱,用戶種類,起日,迄日,系統台,地區,繳別,使用狀態
0,11,16,2023-01-23 15:18:21,1,0,0,0,0.0,CATV,一般收視戶,1999/03/15 00:00:00,NaN,大屯,大里區,3,使用中
1,13,19,2024-08-13 21:53:11,1,0,0,1,0.0,CATV,一般收視戶,2001/11/19 00:00:00,NaN,大屯,大里區,6,使用中
2,28,39,2023-02-15 10:51:09,1,0,0,0,1.0,CATV,一般收視戶,1999/03/01 00:00:00,1999/05/05 00:00:00,大屯,太平區,12,停用
3,47,67,2022-04-15 10:47:15,1,0,0,0,0.0,CATV,一般收視戶,2010/02/23 16:30:17,NaN,大屯,大里區,12,使用中
4,790667,149,2023-03-20 11:37:23,1,0,0,0,0.0,CATV,一般收視戶,1998/10/01 00:00:00,1998/12/31 00:00:00,大屯,大里區,1,停用


In [44]:
ds_doService_combo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300687 entries, 0 to 300686
Data columns (total 16 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   客編      300687 non-null  int64         
 1   相關編號    300687 non-null  int64         
 2   工單日期    300687 non-null  datetime64[ns]
 3   30天內    300687 non-null  int64         
 4   60天內    300687 non-null  int64         
 5   90天內    300687 non-null  int64         
 6   90天以上   300687 non-null  int64         
 7   平均等待天數  300687 non-null  float64       
 8   產品名稱    300687 non-null  object        
 9   用戶種類    300687 non-null  object        
 10  起日      300687 non-null  object        
 11  迄日      9529 non-null    object        
 12  系統台     300687 non-null  object        
 13  地區      300687 non-null  object        
 14  繳別      300687 non-null  int64         
 15  使用狀態    300687 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(7), object(7)
memory usage: 36.7+ 

In [45]:
ds_doService_combo.to_csv('ds_doService_combo.csv', index=False, sep='^')